### Definitions and stuff

In [1]:
using Revise, Plots, Interact, QuadGK, Parameters

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-14482304504618298113\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2499917936249783216\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-14482304504618298113\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
@with_kw struct Units{T<:Real}
    kHz::T = 1000.
    MHz::T = 1.0E6
    GHz::T = 1.0E9
end
const units = Units{Float64}()
@unpack kHz, MHz, GHz = units

Units{Float64}
  kHz: Float64 1000.0
  MHz: Float64 1.0e6
  GHz: Float64 1.0e9


In [3]:
struct LightSource{S<:AbstractString, T<:Real}
    name::S
    df::T
end

In [4]:
const reasonableLaser = LightSource{String,Float64}("Reasonable Laser", 1.0MHz);
const broadLaser = LightSource{String,Float64}("Broad Laser", 5.0GHz);
const broadBand = LightSource{String,Float64}("Broadband Source", 100GHz);

In [5]:
# Lorentz linewidth functions for the source and resonance (resonance width = 1GHz hardcoded)
# (boost is just a multiplicative factor for making numbers more humanly graspable)
const boost = 1.0E6
gL(f::Float64, src::LightSource = reasonableLaser)::Float64 = (boost) * 2/(pi * src.df) * (src.df/2)^2 / (f^2 + (src.df/2)^2);
gRes(f::Float64, df::Float64 = 1GHz)::Float64 = (boost) * 2/(pi * df) * (df/2)^2 / (f^2 + (df/2)^2);

In [6]:
# this is a numerical integration function. Lorentzians above are normalized, result is 1 (within machine precision)
quadgk(f -> (1/boost) * gL(f), -Inf, Inf, atol = 1E-16, rtol = 1E-16, order = 15)[1]

0.9999999999999285

In [7]:
function response(δf::Real, src::LightSource = reasonableLaser)::Float64
    normalizationFactor = 1/quadgk(f -> gRes(f) * gL(f, src), δf-500GHz, δf+500GHz, atol = 1E-4)[1]
    normalizationFactor * quadgk(f -> gRes(f) * gL(f - δf, src), δf-500GHz, δf+500GHz, atol = 1E-4)[1]
end
function transmission(δf::Real, src::LightSource = reasonableLaser, broadband::Bool = false)::Float64
    broadband ? rng = 5*broadBand.df : rng = 50GHz
    1E-6*quadgk(f -> exp(-1000*gRes(f)) * gL(f - δf, src), δf-rng, δf+rng, atol = 1E-4)[1]
end;

In [8]:
function NNsmooth!(y::Array{Float64, 1}, tol::Float64 = 0.01)::Array{Float64, 1}
    for i in 2:1:length(y)
        if abs(y[i] - y[i-1]) > tol
            y[i] = y[i-1]
        end
    end
    return y
end

NNsmooth! (generic function with 2 methods)

In [9]:
function showBroadband()
frng = -500GHz:1GHz:500GHz
tickStep = 100GHz;
ticks = (collect(frng[1]:tickStep:frng[end]), collect(frng[1]:tickStep:frng[end]) ./ GHz)
sliderStep = 5GHz # in GHz;
Yres = gRes.(frng) ./ (2 * gRes(0.0))
@manipulate for _δ = slider(collect(frng[1]:sliderStep:frng[end]) ./ GHz, value = -300, label = "Detuning") |> onchange
    src = broadBand
    Ylas = gL.(frng .- (_δ * GHz), Ref(src)) ./ (2 * gL(0., src))
    YR = 0.5*response.(frng, Ref(src))
    NNsmooth!(YR)
    YT = transmission.(frng, Ref(src), true)
    
    linePlot= plot(frng, [Yres, Ylas], legend = :bottomright,
        xticks = ticks, framestyle = :box, 
        xaxis="Frequency Detuning (GHz)", yaxis="Laser & Resonance", labels = ["Resonance" "Laser"]);
    
    transitionPlot = plot([0,0.25], [0,0], linewidth = 10, linecolor = :green, alpha = 0.5)
    plot!([0, 0.25], [1, 1], linewidth = 10, linecolor = :green, alpha = 0.5)
    plot!([0.125, 0.125], [0, 1 + 0.25E-3 * _δ], line = :arrow, frame = :box, color = (_δ < 0 ? :red : :blue), ticks = nothing, legend = false,
        aspectratio = 0.8, size = (100,300), yaxis=((-0.05, 1.1)), linewidth = 2)
    
    resPlot = plot(frng,YR, alpha = 0.5, label = "Response", legend = :bottomright);
    scatter!([_δ * GHz], [transmission(_δ * GHz, src, true)], label = "");
    scatter!([_δ * GHz], [0.5*response(_δ * GHz, src)], label = "");
    plot!(frng, YT, lims = (-40GHz, 40GHz, 0, 1.05), label = "Transmission", alpha = 0.5,
        xticks = ticks, framestyle = :box, 
        xaxis="Frequency Detuning (GHz)", yaxis=("Response & Transmission", (-0.05, 1.05)));
    
    vbox(src.name, hbox(linePlot, transitionPlot, resPlot))
end
end

showBroadband (generic function with 1 method)

In [10]:
function showBroadLaser()
frng = -50GHz:0.1GHz:50GHz
tickStep = 10GHz;
ticks = (collect(frng[1]:tickStep:frng[end]), collect(frng[1]:tickStep:frng[end]) ./ GHz)
sliderStep = 0.5GHz # in GHz;
Yres = gRes.(frng) ./ (2 * gRes(0.0))
@manipulate for _δ = slider(collect(frng[1]:sliderStep:frng[end]) ./ GHz, value = -30, label = "Detuning") |> onchange
    src = broadLaser
    Ylas = gL.(frng .- (_δ * GHz), Ref(src)) ./ (2 * gL(0., src))
    YR = 0.5*response.(frng, Ref(src))
    #NNsmooth!(YR)
    YT = transmission.(frng, Ref(src))
    
    linePlot= plot(frng, [Yres, Ylas], legend = :bottomright,
        xticks = ticks, framestyle = :box, 
        xaxis="Frequency Detuning (GHz)", yaxis="Laser & Resonance", labels = ["Resonance" "Laser"]);
    
    transitionPlot = plot([0,0.25], [0,0], linewidth = 10, linecolor = :green, alpha = 0.5)
    plot!([0, 0.25], [1, 1], linewidth = 10, linecolor = :green, alpha = 0.5)
    plot!([0.125, 0.125], [0, 1 + 0.25E-2 * _δ], line = :arrow, frame = :box, color = (_δ < 0 ? :red : :blue), ticks = nothing, legend = false,
        aspectratio = 0.8, size = (100,300), yaxis=((-0.05, 1.1)), linewidth = 2)
    
    resPlot = plot(frng,YR, alpha = 0.5, label = "Response", legend = :bottomright);
    scatter!([_δ * GHz], [transmission(_δ * GHz, src)], label = "");
    scatter!([_δ * GHz], [0.5*response(_δ * GHz, src)], label = "");
    plot!(frng, YT, lims = (-40GHz, 40GHz, 0, 1.05), label = "Transmission", alpha = 0.5,
        xticks = ticks, framestyle = :box, 
        xaxis="Frequency Detuning (GHz)", yaxis=("Response & Transmission", (-0.05, 1.05)));
    
    vbox(src.name, hbox(linePlot, transitionPlot, resPlot))
end
end

showBroadLaser (generic function with 1 method)

In [11]:
function showReasonableLaser()
frng = -5GHz:0.01GHz:5GHz
tickStep = 1GHz;
ticks = (collect(frng[1]:tickStep:frng[end]), collect(frng[1]:tickStep:frng[end]) ./ GHz)
sliderStep = 0.1GHz # in GHz;
Yres = gRes.(frng) ./ (2 * gRes(0.0))
@manipulate for _δ = slider(collect(frng[1]:sliderStep:frng[end]) ./ GHz, value = -3, label = "Detuning") |> onchange
    src = reasonableLaser
    Ylas = gL.(frng .- (_δ * GHz), Ref(src)) ./ (2 * gL(0., src))
    YR = 0.5*response.(frng, Ref(src))
    NNsmooth!(YR)
    YT = transmission.(frng, Ref(src))
    
    linePlot= plot(frng, [Yres, Ylas], legend = :bottomright,
        xticks = ticks, framestyle = :box, 
        xaxis="Frequency Detuning (GHz)", yaxis="Laser & Resonance", labels = ["Resonance" "Laser"]);
    
    transitionPlot = plot([0,0.25], [0,0], linewidth = 10, linecolor = :green, alpha = 0.5)
    plot!([0, 0.25], [1, 1], linewidth = 10, linecolor = :green, alpha = 0.5)
    plot!([0.125, 0.125], [0, 1 + 0.25E-1 * _δ], line = :arrow, frame = :box, color = (_δ < 0 ? :red : :blue), ticks = nothing, legend = false,
        aspectratio = 0.8, size = (100,300), yaxis=((-0.05, 1.1)), linewidth = 2)
    
    resPlot = plot(frng,YR, alpha = 0.5, label = "Response", legend = :bottomright);
    scatter!([_δ * GHz], [transmission(_δ * GHz, src)], label = "");
    scatter!([_δ * GHz], [0.5*response(_δ * GHz, src)], label = "");
    plot!(frng, YT, label = "Transmission", alpha = 0.5,
        xticks = ticks, framestyle = :box, 
        xaxis="Frequency Detuning (GHz)", yaxis=("Response & Transmission", (-0.05, 1.05)));
    
    vbox(src.name, hbox(linePlot, transitionPlot, resPlot))
end
end

showReasonableLaser (generic function with 1 method)

### Around a resonance -- with a broadband source, a broad-ish laser, and a reasonable laser

In [54]:
@manipulate for freq_range = slider(500MHz:500MHz:200GHz, value = 500MHz, label="Frequency range"), BroadLaser_amplification = slider(1:100:5001, value = 1.0, label="Broad Laser amplification factor"), BroadBand_amplification = slider(1:100:100001, value = 1.0, label = "Broadband source amplification factor") |> onchange
frng = LinRange(-freq_range, freq_range, 2001)
plot(frng, [gL.(frng), BroadLaser_amplification*gL.(frng,Ref(broadLaser)), BroadBand_amplification*gL.(frng, Ref(broadBand))],
        size = (900,400), label = ["Reasonable laser  | $(reasonableLaser.df/MHz) MHz" "Kinda broad laser  | $(broadLaser.df/GHz) GHz" "Broadband source | just $(broadBand.df/GHz) GHz"], frame = :box,
        xaxis = ("Frequency (Hz)"))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Frequency range"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 400,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\kaltsoplyn\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\kaltsoplyn\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000010b8e290, Task (runnable) @0x0000000010b8e290), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"5.0e8\",\"1.0e9\",\"1.5e9\",\"2.0e9\",\"2.5e9\",\"3.0e9\",\"3.5e9\",\"4.0e9\",\"4.5e9\",\"5.0e9\",\"5.5e9\",\"6.0e9\",\"6.5e9\",\"7.0e9\",\"7.5e9\",\

In [12]:
showBroadband()

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Detuning"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 201,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 2 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
41, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\kaltsoplyn\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\kaltsoplyn\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000012bcce70, Task (runnable) @0x0000000012bcce70), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-500.0\",\"-495.0\",\"-490.0\",\"-485.0\",\"-480.0\",\"-475.0\",\"-470.0\",\"-465.0\",\"-460.0\",\"-455.0\",\"-450.0\",\"-445.0\",\"-440.0\",\"-435.0\",\"-

In [13]:
showBroadLaser()

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Detuning"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 201,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 2 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
41, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\kaltsoplyn\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\kaltsoplyn\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000012bdae10, Task (runnable) @0x0000000012bdae10), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-50.0\",\"-49.5\",\"-49.0\",\"-48.5\",\"-48.0\",\"-47.5\",\"-47.0\",\"-46.5\",\"-46.0\",\"-45.5\",\"-45.0\",\"-44.5\",\"-44.0\",\"-43.5\",\"-43.0\",\"-42.5

In [14]:
showReasonableLaser()

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Detuning"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 101,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 2 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
21, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\kaltsoplyn\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\kaltsoplyn\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\kaltsoplyn\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000010b8c8b0, Task (runnable) @0x0000000010b8c8b0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-5.0\",\"-4.9\",\"-4.8\",\"-4.7\",\"-4.6\",\"-4.5\",\"-4.4\",\"-4.3\",\"-4.2\",\"-4.1\",\"-4.0\",\"-3.9\",\"-3.8\",\"-3.7\",\"-3.6\",\"-3.5\",\"-3.4\",\"-3